# **Clustering - Mall Customer Segmentation**

Clustering is the task of dividing the data points into a number of groups. Data points in the same groups are more similar to other data points in the same group and dissimilar to the data points in other groups.

Malls and other shopping places often compete to increase their customers and gain greater profits. Mall owners utilize their basic customer membership data and develop machine learning models to understand the purchasing behavior and set the right target customers so that the sense can be given to marketing team and plan the strategy accordingly.

The customer data consist of the following features :
1. CustomerID: It is the unique ID given to a customer
2. Gender: Gender of the customer
3. Age: The age of the customer
4. Annual Income(k$): It is the annual income of the customer
5. Spending Score: It is the score(out of 100) given to a customer by the mall authorities, based on the money spent and the behavior of the customer.

Source: https://www.kaggle.com/vjchoudhary7/customer-segmentation-tutorial-in-python


## **Install and Import Libraries**

***Install Library***

In [ ]:
# Install Category Encoders
! pip install category_encoders

In [ ]:
!pip install kneed 
from kneed import DataGenerator, KneeLocator 

***Import Libraries***

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

import matplotlib.cm as cm
from scipy import stats

import os
import warnings
warnings.filterwarnings("ignore")

## **Import Data**

***Mall Customer Data***

In [ ]:
#Import Dataset
from google.colab import files
uploaded = files.upload()


In [ ]:
#import data to Google Colab 
import pandas as pd 
df_customer = pd.read_csv('mall_customer.csv', sep=';')

In [ ]:
#Check our dataset 
df_customer

In [ ]:
# Prints the Dataset Information
df_customer.info()

In [ ]:
#rename head of Dataset 
df_customer.rename(index=str, columns={'ann_income_kUSD': 'income',
                              'spending_score': 'score'}, inplace=True)
df_customer.head()

In [ ]:
# Prints Descriptive Statistics
df_customer.describe().transpose()

## **Explore the Dataset**

Data stratified by gender - the only categorical variable.

In [ ]:
# subset with males age
males_age = df_customer[df_customer['gender']=='male']['age']
# subset with females age 
females_age = df_customer[df_customer['gender']=='female']['age']
age_bins = range  (15,75,5)

In [ ]:
#males Histogram 
fig2, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,5), sharey=True)
sns.distplot(males_age, bins=age_bins, kde=False, color='#0066ff', ax=ax1, hist_kws=dict(edgecolor="k", linewidth=2))
ax1.set_xticks(age_bins)
ax1.set_ylim(top=25)
ax1.set_title('Males')
ax1.set_ylabel('Count')
ax1.text(45,23, "TOTAL count: {}".format(males_age.count()))
ax1.text(45,22, "Mean age: {:.1f}".format(males_age.mean()))

# female Histogram 
sns.distplot(females_age, bins=age_bins, kde=False, color='#cc66ff', ax=ax2, hist_kws=dict(edgecolor="k", linewidth=2))
ax2.set_xticks(age_bins)
ax2.set_title('Females')
ax2.set_ylabel('Count')
ax2.text(45,23, "TOTAL count: {}".format(females_age.count()))
ax2.text(45,22, "Mean age: {:.1f}".format(females_age.mean()))

plt.show()


In [ ]:
print('Kolgomorov-Smirnov test p-value: {:.2f}'.format(stats.ks_2samp(males_age, females_age)[1]))

The average age of male customers is lightly higher than female ones (39.8 versus 38.1). Distribution of male age is more uniform than females, where we can observe that the biggest age group is 30-35 years old. Kolgomorov-Smirnov test shows that the differences between these two groups are statistically insignificant.

In [ ]:
def labeler(pct, allvals):
    absolute = int(pct/100.*np.sum(allvals))
    return "{:.1f}%\n({:d})".format(pct, absolute)

sizes = [males_age.count(),females_age.count()] # wedge sizes

fig0, ax1 = plt.subplots(figsize=(6,6))
wedges, texts, autotexts = ax1.pie(sizes,
                                   autopct=lambda pct: labeler(pct, sizes),
                                   radius=1,
                                   colors=['#0066ff','#cc66ff'],
                                   startangle=90,
                                   textprops=dict(color="w"),
                                   wedgeprops=dict(width=0.7, edgecolor='w'))

ax1.legend(wedges, ['male','female'],
           loc='center right',
           bbox_to_anchor=(0.7, 0, 0.5, 1))

plt.text(0,0, 'TOTAL\n{}'.format(df_customer['age'].count()),
         weight='bold', size=12, color='#52527a',
         ha='center', va='center')

plt.setp(autotexts, size=12, weight='bold')
ax1.axis('equal')  # Equal aspect ratio
plt.show()

There are slightly more female customers than male ones (112 vs. 87). Females are 56% of total customers.

In [ ]:
males_income = df_customer[df_customer['gender']=='male']['income'] # subset with males income
females_income = df_customer[df_customer['gender']=='female']['income'] # subset with females income

my_bins = range(10,150,10)

# males histogram
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18,5))
sns.distplot(males_income, bins=my_bins, kde=False, color='#0066ff', ax=ax1, hist_kws=dict(edgecolor="k", linewidth=2))
ax1.set_xticks(my_bins)
ax1.set_yticks(range(0,24,2))
ax1.set_ylim(0,22)
ax1.set_title('Males')
ax1.set_ylabel('Count')
ax1.text(85,19, "Mean income: {:.1f}".format(males_income.mean()))
ax1.text(85,18, "Median income: {:.1f}".format(males_income.median()))
ax1.text(85,17, "Std. deviation: {:.1f}".format(males_income.std()))

# females histogram
sns.distplot(females_income, bins=my_bins, kde=False, color='#cc66ff', ax=ax2, hist_kws=dict(edgecolor="k", linewidth=2))
ax2.set_xticks(my_bins)
ax2.set_yticks(range(0,24,2))
ax2.set_ylim(0,22)
ax2.set_title('Females')
ax2.set_ylabel('Count')
ax2.text(85,19, "Mean income: {:.1f}".format(females_income.mean()))
ax2.text(85,18, "Median income: {:.1f}".format(females_income.median()))
ax2.text(85,17, "Std. deviation: {:.1f}".format(females_income.std()))

# boxplot
sns.boxplot(x='gender', y='income', data=df_customer, ax=ax3)
ax3.set_title('Boxplot of annual income')
plt.show()

In [ ]:
print('Kolgomorov-Smirnov test p-value: {:.2f}'.format(stats.ks_2samp(males_income, females_income)[1]))

Mean income of males is higher than females 62.2  vs. 59.2 
Median income  of male customers 62.5 is higher than female ones 60
 
Standard deviation is similar for both groups. There is one outlier in male group with an annual income of about 140 . K-S test shows that these two groups are not statistically different.  

In [ ]:
males_spending = df_customer[df_customer ['gender']=='male']['score'] # subset with males age
females_spending = df_customer[df_customer['gender']=='female']['score'] # subset with females age

spending_bins = range(0,105,5)

# males histogram
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18,5))
sns.distplot(males_spending, bins=spending_bins, kde=False, color='#0066ff', ax=ax1, hist_kws=dict(edgecolor="k", linewidth=2))
ax1.set_xticks(spending_bins)
ax1.set_xlim(0,100)
ax1.set_yticks(range(0,17,1))
ax1.set_ylim(0,16)
ax1.set_title('Males')
ax1.set_ylabel('Count')
ax1.text(50,15, "Mean spending score: {:.1f}".format(males_spending.mean()))
ax1.text(50,14, "Median spending score: {:.1f}".format(males_spending.median()))
ax1.text(50,13, "Std. deviation score: {:.1f}".format(males_spending.std()))

# females histogram
sns.distplot(females_spending, bins=spending_bins, kde=False, color='#cc66ff', ax=ax2, hist_kws=dict(edgecolor="k", linewidth=2))
ax2.set_xticks(spending_bins)
ax2.set_xlim(0,100)
ax2.set_yticks(range(0,17,1))
ax2.set_ylim(0,16)
ax2.set_title('Females')
ax2.set_ylabel('Count')
ax2.text(50,15, "Mean spending score: {:.1f}".format(females_spending.mean()))
ax2.text(50,14, "Median spending score: {:.1f}".format(females_spending.median()))
ax2.text(50,13, "Std. deviation score: {:.1f}".format(females_spending.std()))

# boxplot
sns.boxplot(x='gender', y='score', data=df_customer, ax=ax3)
ax3.set_title('Boxplot of spending score')
plt.show()

plt.show()

In [ ]:
print('Kolgomorov-Smirnov test p-value: {:.2f}'.format(stats.ks_2samp(males_spending, females_spending)[1]))

A mean spending score for women (51.5) is higher than men (48.5). The K-S test p-value indicates that there is no evidence to reject the null-hypothesis, however the evidence is not so strong as in previous comparisons.  

In [ ]:
#Calculate The Median Income for all age groups 
medians_by_age_group = df_customer.groupby(["gender",pd.cut(df_customer['age'], age_bins)]).median()
medians_by_age_group.index = medians_by_age_group.index.set_names(['gender', 'age_group'])
medians_by_age_group.reset_index(inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
sns.barplot(x='age_group', y='income', hue='gender', data=medians_by_age_group,
            palette=['#cc66ff','#0066ff'],
            alpha=0.7,edgecolor='k',
            ax=ax)
ax.set_title('Median annual income of male and female customers')
ax.set_xlabel('Age group')
plt.show()

the most wealthy customers are in age of 25-45 years old. The biggest difference between women and men is visible in age groups 25-30 (male more rich) and 50-55 (female more rich).

## CLUSTERING

***Visualize Data using Pairplot***

In [ ]:
# Let's see our data in a detailed way with pairplot
X = df_customer.drop(['customer', 'gender'], axis=1)
sns.pairplot(df_customer.drop('customer', axis=1), hue='gender', aspect=1.5)
plt.show()

***Visualize Data using Scatterplot***

In [ ]:
# Draw Scatter Plot
sns.relplot(x='income', y='score', hue='gender', size='age', kind='scatter', col='gender', data=df_customer)
plt.title('Customer Behavior')
plt.xlabel('Annual Income k USD')
plt.ylabel('Spending Score')

***Visualize Correlation between Features***

In [ ]:
# Draw Correlation to see hierarchically clustered heatmap
sns.clustermap(df_customer.corr(), center=0, cmap='vlag', linewidths=.75)

## **Data Preprocessing**

***Handling Missing Values***

In [ ]:
# Check for Missing Values
df_customer.isnull().sum()

In [ ]:
#datashape ( we have 200 list of data with 5 data head)
df_customer.shape


In [ ]:
#selecting feature (use iloc iloc as integer index-based. So here, we have to specify rows and columns by their integer index.)
# 3 column of income, 4 column of score 
X_numerics = df_customer.iloc[:,[3,4]].values 

In [ ]:
Z_numerics = df_customer[['income', 'score']] # subset with numerical only

In [ ]:
Z_numerics

## K-MEANS 

In [ ]:
from sklearn.metrics import silhouette_score
n_clusters = [2,3,4,5,6,7,8,9,10] # number of clusters
clusters_inertia = [] # inertia of clusters
s_scores = [] # silhouette scores

for n in n_clusters:
    KM_est = KMeans(n_clusters=n, init='k-means++').fit(Z_numerics)
    clusters_inertia.append(KM_est.inertia_)    # data for the elbow method
    silhouette_avg = silhouette_score(Z_numerics, KM_est.labels_)
    s_scores.append(silhouette_avg) # data for the silhouette score method

In [ ]:
#Determine the optimal number of cluster 
# Elbow Method
from sklearn.cluster import KMeans
wcss = []
for i in range(1,11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(Z_numerics)
    wcss.append(kmeans.inertia_)
kl = KneeLocator(
    range(1,11), wcss, curve="convex", direction= "decreasing")
kl.elbow 
    

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
ax = sns.lineplot(n_clusters, clusters_inertia, marker='o', ax=ax)
ax.set_title("Elbow method")
ax.set_xlabel("number of clusters")
ax.set_ylabel("clusters inertia")
ax.axvline(5, ls="--", c="red")
plt.grid()
plt.show()

Let's see the silhouette score

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
ax = sns.lineplot(n_clusters, s_scores, marker='o', ax=ax)
ax.set_title("Silhouette score method")
ax.set_xlabel("number of clusters")
ax.set_ylabel("Silhouette score")
ax.axvline(5, ls="--", c="red")
plt.grid()
plt.show()

In [ ]:
x = df_customer.iloc[:,[3,4]].values 

In [ ]:
# Silhouette score (study the separation distance between the resulting clusters) 
# how close each point in one cluster is to points in the neighboring clusters 
# thus provides a way to assess parameters like number of clusters visually
import numpy as np

for n_clusters in range(2,11):
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(x) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(x)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(x, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(x, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(x[:, 0], x[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                c=colors, edgecolor='k')

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                c="white", alpha=1, s=200, edgecolor='k')

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

plt.show()

Silhouette score method indicates the best options would be 5 clusters. 

In [ ]:
#selecting feature (use iloc iloc as integer index-based. So here, we have to specify rows and columns by their integer index.)
# 2 Age, 3 column of income, 4 column of score 
Z_numerics = df_customer.iloc[:,[2,3,4]].values 

In [ ]:
Z_numerics = df_customer[['age','income', 'score']] # subset with numerical only

In [ ]:
KM_clusters = KMeans(n_clusters=5, init='k-means++').fit(Z_numerics) # initialise and fit K-Means model

KM_clustered = Z_numerics.copy()
KM_clustered.loc[:,'Cluster'] = KM_clusters.labels_ # append labels to points

In [ ]:
fig1, (axes) = plt.subplots(1,2,figsize=(12,5))


scat_1 = sns.scatterplot('income', 'score', data=KM_clustered,
                hue='Cluster', ax=axes[0], palette='Set1', legend='full')

sns.scatterplot('age', 'score', data=KM_clustered,
                hue='Cluster', palette='Set1', ax=axes[1], legend='full')

axes[0].scatter(KM_clusters.cluster_centers_[:,1],KM_clusters.cluster_centers_[:,2], marker='s', s=40, c="blue")
axes[1].scatter(KM_clusters.cluster_centers_[:,0],KM_clusters.cluster_centers_[:,2], marker='s', s=40, c="blue")
plt.show()

K-Means algorithm generated the following 5 clusters:

0 clients with **medium** annual income and **medium** spending score --- 
1 clients with **high** annual income and **low** spending score ---
2 clients with **high** annual income and **high** spending score ---
3 clients with **low** annual income and **high** spending score ---
4 clients with **low** annual income and **low** spending score ---
There are no distinct groups is terms of customers age.

In [ ]:
#Sizes of the clusters:
KM_clust_sizes = KM_clustered.groupby('Cluster').size().to_frame()
KM_clust_sizes.columns = ["KM_size"]
KM_clust_sizes

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(7, 7))
ax = Axes3D(fig, rect=[0, 0, .99, 1], elev=20, azim=210)
ax.scatter(KM_clustered['age'],
           KM_clustered['income'],
           KM_clustered['score'],
           c=KM_clustered['Cluster'],
           s=35, edgecolor='k', cmap=plt.cm.Set1)

ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])
ax.set_xlabel('Age')
ax.set_ylabel('Annual Income')
ax.set_zlabel('Spending Score')
ax.set_title('3D view of K-Means 5 clusters')
ax.dist = 12

plt.show()

In [ ]:
# Below a Plotly version:
import plotly as py
import plotly.graph_objs as go

def tracer(db, n, name):
    '''
    This function returns trace object for Plotly
    '''
    return go.Scatter3d(
        x = db[db['Cluster']==n]['age'],
        y = db[db['Cluster']==n]['score'],
        z = db[db['Cluster']==n]['income'],
        mode = 'markers',
        name = name,
        marker = dict(
            size = 5
        )
     )

trace0 = tracer(KM_clustered, 0, 'Cluster 0')
trace1 = tracer(KM_clustered, 1, 'Cluster 1')
trace2 = tracer(KM_clustered, 2, 'Cluster 2')
trace3 = tracer(KM_clustered, 3, 'Cluster 3')
trace4 = tracer(KM_clustered, 4, 'Cluster 4')

data = [trace0, trace1, trace2, trace3, trace4]

layout = go.Layout(
    title = 'Clusters by K-Means',
    scene = dict(
            xaxis = dict(title = 'Age'),
            yaxis = dict(title = 'Spending Score'),
            zaxis = dict(title = 'Annual Income')
        )
)

fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig)

## DBSCAN 

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
#first create a matrix of investigated combinations.
from itertools import product

eps_values = np.arange(8,12.75,0.25) # eps values to be investigated
min_samples = np.arange(3,10) # min_samples values to be investigated
DBSCAN_params = list(product(eps_values, min_samples))

In [ ]:
#Colecting number of generated clusters.
no_of_clusters = []
sil_score = []

for p in DBSCAN_params:
    DBS_clustering = DBSCAN(eps=p[0], min_samples=p[1]).fit(Z_numerics)
    no_of_clusters.append(len(np.unique(DBS_clustering.labels_)))
    sil_score.append(silhouette_score(Z_numerics, DBS_clustering.labels_))

In [ ]:
#A heatplot to shows how many clusters were genreated by the algorithm for the respective parameters combinations.
tmp = pd.DataFrame.from_records(DBSCAN_params, columns =['Eps', 'Min_samples'])   
tmp['No_of_clusters'] = no_of_clusters

pivot_1 = pd.pivot_table(tmp, values='No_of_clusters', index='Min_samples', columns='Eps')

fig, ax = plt.subplots(figsize=(12,6))
sns.heatmap(pivot_1, annot=True,annot_kws={"size": 16}, cmap="YlGnBu", ax=ax)
ax.set_title('Number of clusters')
plt.show()

the number of clusters vary from 17 to 4.

In [ ]:
tmp = pd.DataFrame.from_records(DBSCAN_params, columns =['Eps', 'Min_samples'])   
tmp['Sil_score'] = sil_score

pivot_1 = pd.pivot_table(tmp, values='Sil_score', index='Min_samples', columns='Eps')

fig, ax = plt.subplots(figsize=(18,6))
sns.heatmap(pivot_1, annot=True, annot_kws={"size": 10}, cmap="YlGnBu", ax=ax)
plt.show()

Global maximum is 0.26 for eps=12.5 and min_samples=4.

In [ ]:
DBS_clustering = DBSCAN(eps=12.5, min_samples=4).fit(Z_numerics)

DBSCAN_clustered = Z_numerics.copy()
DBSCAN_clustered.loc[:,'Cluster'] = DBS_clustering.labels_ # append labels to points

In [ ]:
DBSCAN_clust_sizes = DBSCAN_clustered.groupby('Cluster').size().to_frame()
DBSCAN_clust_sizes.columns = ["DBSCAN_size"]
DBSCAN_clust_sizes

DBSCAN created 5 clusters plus outliers cluster (-1). Sizes of clusters 0-4 vary significantly - some have only 4 or 8 observations. There are 18 outliers.

In [ ]:
outliers = DBSCAN_clustered[DBSCAN_clustered['Cluster']==-1]

fig2, (axes) = plt.subplots(1,2,figsize=(12,5))


sns.scatterplot('income', 'score',
                data=DBSCAN_clustered[DBSCAN_clustered['Cluster']!=-1],
                hue='Cluster', ax=axes[0], palette='Set1', legend='full', s=45)

sns.scatterplot('age', 'score',
                data=DBSCAN_clustered[DBSCAN_clustered['Cluster']!=-1],
                hue='Cluster', palette='Set1', ax=axes[1], legend='full', s=45)

axes[0].scatter(outliers['income'], outliers['score'], s=5, label='outliers', c="k")
axes[1].scatter(outliers['age'], outliers['score'], s=5, label='outliers', c="k")
axes[0].legend()
axes[1].legend()
plt.setp(axes[0].get_legend().get_texts(), fontsize='10')
plt.setp(axes[1].get_legend().get_texts(), fontsize='10')

plt.show()

The graph above shows that there are some outliers - these points do not meet distance and minimum samples requirements to be recognised as a cluster.

## COMPARISON 


In [ ]:
fig1.suptitle('K-Means', fontsize=16)
fig1

In [ ]:
fig2.suptitle('DBSCAN', fontsize=16)
fig2

In [ ]:
clusters = pd.concat([KM_clust_sizes, DBSCAN_clust_sizes],axis=1, sort=False)
clusters

***Data Standardization***

In [ ]:
# Importing Standardscalar Module 
from sklearn.preprocessing import StandardScaler 

# Set Name for StandardScaler as scaler
scaler = StandardScaler() 

# Select Data
df_standardized = df_customer[['age',	'income',	'score']]

# Fit Standardization
column_names = df_standardized.columns.tolist()
df_standardized[column_names] = scaler.fit_transform(df_standardized[column_names])
df_standardized.sort_index(inplace=True)
df_standardized

## Hierarchical Clustering (visualization)

In [ ]:
# Modeling and Visualizing Clusters by Dendogram
import scipy.cluster.hierarchy as sch
dend = sch.dendrogram(sch.linkage(df_standardized, method='ward'))
plt.title('Dendrogram')
plt.xlabel('annual income')
plt.ylabel('spending score')
plt.rcParams["figure.figsize"] = [8,4]
plt.show()

## Partitional Clustering (Visualization)

In [ ]:
fig1, (axes) = plt.subplots(1,2,figsize=(12,5))


scat_1 = sns.scatterplot('income', 'score', data=KM_clustered,
                hue='Cluster', ax=axes[0], palette='Set1', legend='full')

sns.scatterplot('age', 'score', data=KM_clustered,
                hue='Cluster', palette='Set1', ax=axes[1], legend='full')

axes[0].scatter(KM_clusters.cluster_centers_[:,1],KM_clusters.cluster_centers_[:,2], marker='s', s=40, c="blue")
axes[1].scatter(KM_clusters.cluster_centers_[:,0],KM_clusters.cluster_centers_[:,2], marker='s', s=40, c="blue")
plt.show()

## Density based Clustering (Visualization) 

In [ ]:
tmp = pd.DataFrame.from_records(DBSCAN_params, columns =['Eps', 'Min_samples'])   
tmp['Sil_score'] = sil_score

pivot_1 = pd.pivot_table(tmp, values='Sil_score', index='Min_samples', columns='Eps')

fig, ax = plt.subplots(figsize=(18,6))
sns.heatmap(pivot_1, annot=True, annot_kws={"size": 10}, cmap="YlGnBu", ax=ax)
plt.show()